In [1]:
import time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

如果想要使用 Python 來下載網頁上的資料，最基本的作法就是以 requests 模組建立適當的 HTTP 請求，透過 HTTP 請求從網頁伺服器下載指定的資料。
普通單純的網頁，只需要用最簡單的 GET 請求即可直接下載。

# 伺服器回應的狀態碼
print(r.status_code)

如果顯示 200 就代表沒問題。我們也可以利用以下這個判斷式來檢查狀態碼：

# 檢查狀態碼是否 OK
if r.status_code == requests.codes.ok:
  print("OK")

r.text 可以查看原始的網頁 HTML 程式碼。

In [2]:
url = 'https://racing.appledaily.com.hk/race-day/race-position?raceDay=2688&race=23731'
response = requests.get(url) #, headers = {'user-agent':'Mozilla/5.0'}
# r  You can merely print r to see if the response is 200, 200 means ok

In [3]:
#response.text  #The result only shows part of the html source code....the part i want is not gotten...我嘗試在request到加user-agent但不成功


從網絡原始碼取出所要資訊就要用BeautifulSoup. 
Beautiful Soup is a Python library for pulling data out of HTML and XML files.

# Making the soup
To parse a document, pass it into the BeautifulSoup constructor. 
First, the document is converted to Unicode, and HTML entities are converted to Unicode characters
Beautiful Soup then parses the document using the best available parser. It will use an HTML parser unless you specifically tell it to use an XML parser.
source: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser 

In [4]:
soup = BeautifulSoup(response.content, 'lxml') #There are other parser such as lxml's HTML parser and lxml's XML parser. 
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   賽事走位重播及平面圖 | 蘋果馬網
  </title>
  <meta content="各場賽事賽馬結果，沿途走位重播、走位圖及平面圖、馬匹走位、分段時間、總時間紀錄、勝負距離及賽後報告等，幫馬迷作賽後檢討，做定功課鎖定備忘馬匹，準備下次出擊。" name="description"/>
  <meta content="賽馬結果,賽事重播,走位圖,平面圖,落飛,賠率,馬匹走位,分段時間,總時間,勝負距離,賽後報告,賽後重溫,蘋果馬經,賽馬,馬會" name="keywords"/>
  <meta content="YES" name="ssr-check"/>
  <link href="/front.css" rel="stylesheet"/>
  <script src="/js/front.js?33955cfcda60f8379bb2">
  </script>
  <script src="/js/vendors~front.ac01469ea8c1f54097d1.js?ac01469ea8c1f54097d1">
  </script>
  <script src="https://staticlayout.appledaily.hk/uvp2/js/anvp.common.lib-v1.0.3.js?v=20180119">
  </script>
  <script async="async" src="https://www.googletagservices.com/tag/js/gpt.js">
  </script>
  <script src="https://static.omoplanet.com/WebSDK/v3.3.1/omo-sdk-3.3.1.min.js">
  </script>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
  new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[

我地想要"https://racing.appledaily.com.hk/race-day/race-position?raceDay=XXXX&race=XXXX" XXXX既部份，因為XXXX這些是對應不同日期乃至第幾場既資料，所以我地可以係define左既url 到試下搵。(因為個網頁有整左清單同埋按鈕俾你去其他網頁，代表其實你可以係一個網頁入面抓取去其他網要所需既資訊)。如下圖係個清單到right click 檢查元素，可以帶到我地去所對應既tag, which is "select" tag。


soup.find_all():係beautifulsoup object 既method?
find_all(name, attrs, recursive, string, limit, **kwargs)
looks through a tag’s descendants and retrieves all descendants that match your filters.

Question: 
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup.find_all('select')
但出唔到野，上網睇有人懷疑因為 the elements are rendered using javascript AFTER the page has loaded.
咁所以request.get 限制係咩? 點睇幾時用左javascript?

In [5]:
soup.find_all(attrs={'value':(\d*)})

SyntaxError: unexpected character after line continuation character (<ipython-input-5-d866535079d4>, line 1)

In [ ]:
#有無用? 參考https://stackoverflow.com/questions/42147601/pyqt4-to-pyqt5-mainframe-deprecated-need-fix-to-load-web-pages 
#參考https://impythonist.wordpress.com/2015/01/06/ultimate-guide-for-scraping-javascript-rendered-web-pages/
import sys 
from PyQt5.QtWebEngineWidgets import QWebEnginePage
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl

class Render(QWebEnginePage):  
  def __init__(self, url):  
    self.app = QApplication(sys.argv)  
    QWebEnginePage.__init__(self)  
    self.html = ''
    self.loadFinished.connect(self._on_load_finished)  
    self.load(QUrl(url))  
    self.app.exec_()  
    
  def _on_load_finished(self):
    self.html = self.toHtml(self.Callable)
    print('Load finished')

  def Callable(self, html_str):
    self.html = html_str
    self.app.quit()  